<a href="https://colab.research.google.com/github/daizybhadresha/python_practice/blob/main/MCFS_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
## Loading the data

import pandas as pd

path1 = '/content/drive/My Drive/Capstone/capstone_data/Customer_Data.xlsx'
path2 = '/content/drive/My Drive/Capstone/capstone_data/Plant Master.xlsx'
path3 = '/content/drive/My Drive/Capstone/capstone_data/Final_invoice.csv'
path4 = '/content/drive/My Drive/Capstone/capstone_data/JTD.csv'

cust_df = pd.read_excel(path1)
plant_df = pd.read_excel(path2)
invoice_df = pd.read_csv(path3)
jtd_df = pd.read_csv(path4)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,10,14,15,21,41,51) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
# Deleting all the columns with all NaN values
def delCols_With_AllNaN(dataframe):
  dataframe_1 = dataframe.dropna(axis = 1, how='all')
  print("Deleted columns: ",list(set(dataframe.columns) - set(dataframe_1.columns)))
  print("Dimension after Deletions: ", dataframe_1.shape)
  return dataframe_1

# Deleting all the columns with all NaN values
def delRows_With_AnyNaN(dataframe):
  dataframe_1 = dataframe.dropna(axis = 0, how='any')
  print("Dimension after Deletions: ", dataframe_1.shape)
  return dataframe_1

# Removing the cols which only has 2 values - 0's/NaN
def delCols_With_AllZeroes(dataframe):
  numeric_cols = dataframe.select_dtypes('number').columns
  cols_to_be_deleted = []
  for col in numeric_cols:
    if dataframe[col].sum() == 0:
      cols_to_be_deleted += [col]
  
  print(cols_to_be_deleted)
  dataframe_1 = dataframe.drop(columns=cols_to_be_deleted)
  print("Deleted columns: ",list(set(dataframe.columns) - set(dataframe_1.columns)))
  print("Dimension after Deletions: ", dataframe_1.shape)
  return dataframe_1

# Removing unwanted columns/ with > **% missing data
def delCols_With_MissingThreshold(dataframe,threshold):
  l = []
  dataframe_1 = dataframe.drop(dataframe.loc[:,list((100*(dataframe.isnull().sum()/len(dataframe.index))>=threshold))].columns, 1)
  l = list(dataframe_1.columns.values)
  print("# Columns having more than %s percent missing values:"%threshold,(dataframe.shape[1] - len(l)))
  print("Columns deleted: ",list(set(list((dataframe.columns.values))) - set(l)))
  print("Dimension after Deletions: ", dataframe_1.shape)
  return dataframe_1

# Fill null with 'No Info.'
def fillCol_With_Text_NoInfo(dataframe, cols):
  for col in cols:
    dataframe[col].fillna('No Info.', inplace = True)



In [ ]:
#### Cleaning of invoice df 

# 1st - Deleting all the columns with all NaN values
invoice_df_1 = delCols_With_AllNaN(invoice_df)

# 2nd - Removing the cols which only has 2 values - 0's/NaN 
invoice_df_2 = delCols_With_AllZeroes(invoice_df_1)

# 3rd - Removing unwanted columns/ with >90% missing data
invoice_df_2 = delCols_With_MissingThreshold(invoice_df_2, 90)

# 4th - Deleting the unrequired columns
cols_to_be_deleted = ['Gate Pass Time', 'ODN No.',  'Print Status', 'Recovrbl Exp', 'Regn No', 'User ID', 'Total Value']
invoice_df_2 = invoice_df_2.drop(columns=cols_to_be_deleted)
print("Dimension after Deletions: ", invoice_df_2.shape)

# 5th - Missing Technician - 'No Info.'
fillCol_With_Text_NoInfo(invoice_df_2, ['Technician Name'])

print(invoice_df_2.columns, invoice_df_2.shape)


Deleted columns:  ['Service Advisor Name']
Dimension after Deletions:  (492314, 58)
['Amt Rcvd From Custom', 'Amt Rcvd From Ins Co', 'CGST(14%)', 'CGST(2.5%)', 'CGST(6%)', 'CGST(9%)', 'IGST(12%)', 'IGST(18%)', 'IGST(28%)', 'IGST(5%)', 'Outstanding Amt', 'SGST/UGST(14%)', 'SGST/UGST(2.5%)', 'SGST/UGST(6%)', 'SGST/UGST(9%)', 'TDS amount', 'Total CGST', 'Total GST', 'Total IGST', 'Total SGST/UGST']
Deleted columns:  ['TDS amount', 'IGST(12%)', 'Total SGST/UGST', 'Amt Rcvd From Custom', 'SGST/UGST(6%)', 'IGST(18%)', 'SGST/UGST(9%)', 'CGST(9%)', 'SGST/UGST(2.5%)', 'Outstanding Amt', 'IGST(5%)', 'CGST(2.5%)', 'Total GST', 'SGST/UGST(14%)', 'CGST(6%)', 'Total IGST', 'IGST(28%)', 'Amt Rcvd From Ins Co', 'Total CGST', 'CGST(14%)']
Dimension after Deletions:  (492314, 38)
# Columns having more than 90 percent missing values: 6
Columns deleted:  ['Cash /Cashless Type', 'Policy no.', 'Gate Pass Date', 'Expiry Date', 'Insurance Company', 'Claim No.']
Dimension after Deletions:  (492314, 32)
Dimensi

In [ ]:
## Imputing Area/Locality - Creating new Locality Dataset ["Locality name", "Pincode"]

path5 = '/content/drive/My Drive/Capstone/capstone_data/Location.csv'
locality_df = pd.read_csv(path5)
locality_df['Pincode'] = locality_df.key.apply(lambda x : x[3:])
locality_df.drop_duplicates(subset ="Pincode", keep = 'first', inplace = True)
locality_df.rename(columns={"place_name": "Locality name"}, inplace = True)
locality_df = locality_df[["Locality name","Pincode"]]

path6 = '/content/drive/My Drive/Capstone/capstone_data/Locality_New.csv'
locality_df_1 = pd.read_csv(path6)
locality_df_1.drop_duplicates(subset ="Pincode", keep = 'first', inplace = True)
locality_df_1 = locality_df_1[["Locality name","Pincode"]]

path7 = '/content/drive/My Drive/Capstone/capstone_data/Locality_2.csv'
locality_df_2 = pd.read_csv(path7)
locality_df_2.drop_duplicates(subset ="pincode", keep = 'first', inplace = True)
locality_df_2.rename(columns={"Taluk": "Locality name", "pincode":"Pincode" }, inplace = True)
locality_df_2 = locality_df_2[["Locality name","Pincode"]]

area_df = pd.concat([locality_df, locality_df_1, locality_df_2])
#print(len(area_df))
area_df.drop_duplicates(subset ="Pincode", keep = 'first', inplace = True)
print("Total unique rows of Locality df : ", len(area_df))
print(area_df.head())

## Missing Area/Locality
print("Total missing area/locality : ",invoice_df_2['Area / Locality'].isnull().sum())   # 23890,  4.85% missing

## Merging the newly created locality dataset with the current Invoice dataset
result = pd.merge(invoice_df_2, area_df, how='left', left_on='Pin code', right_on='Pincode', indicator=True)  #pd.merge(invoice_df_2, locality_df, on='Pin code', how='left', indicator=True)
print(result.columns)
result['Area / Locality'].fillna(result['Locality name'], inplace = True)
print("After merge not null rows : ", result['Area / Locality'].notnull().sum())  # 15,522‬,  3.15% missing

## Filling NaN with "No Info."
fillCol_With_Text_NoInfo(result, ['Area / Locality'])
print("Total Not null rows : ",result['Area / Locality'].notnull().sum())
result.drop(['Locality name', 'Pincode', '_merge'],axis=1, inplace = True)
print(result.columns)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (2,6,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Total unique rows of Locality df :  19977
             Locality name Pincode
0          Connaught Place  110001
1               Darya Ganj  110002
2                  Aliganj  110003
3       Rashtrapati Bhawan  110004
4  Lower Camp Anand Parbat  110005
Total missing area/locality :  23890
Index(['Unnamed: 0', 'Area / Locality', 'CITY', 'Cust Type', 'Customer No.',
       'District', 'Invoice Date', 'Invoice No', 'Invoice Time', 'Job Card No',
       'JobCard Date', 'JobCard Time', 'KMs Reading', 'Labour Total', 'Make',
       'Misc Total', 'Model', 'OSL Total', 'Order Type', 'Parts Total',
       'Pin code', 'Plant', 'Plant Name1', 'Technician Name',
       'Total Amt Wtd Tax.', 'Locality name', 'Pincode', '_merge'],
      dtype='object')
After merge not null rows :  476792
Total Not null rows :  492314
Index(['Unnamed: 0', 'Area / Locality', 'CITY', 'Cust Type', 'Customer No.',
       'District', 'Invoice Date', 'Invoice No', 'Invoice Time', 'Job Card No',
       'JobCard Date', 'JobCa

In [ ]:
result.drop(result.columns[result.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
print(result.columns)

Index(['Area / Locality', 'CITY', 'Cust Type', 'Customer No.', 'District',
       'Invoice Date', 'Invoice No', 'Invoice Time', 'Job Card No',
       'JobCard Date', 'JobCard Time', 'KMs Reading', 'Labour Total', 'Make',
       'Misc Total', 'Model', 'OSL Total', 'Order Type', 'Parts Total',
       'Pin code', 'Plant', 'Plant Name1', 'Technician Name',
       'Total Amt Wtd Tax.'],
      dtype='object')


In [ ]:
## Exporting processed data

result.to_csv('/content/drive/My Drive/Capstone/capstone_data/Invoice_Preprocesed.csv')


In [ ]:
#### To reformat


## No. of states

print(plant_df['State'].unique())
print(invoice_df['District'].unique())

### Proof that there are districts in Invoice whose pincode does not match with pincode in plant dataset   Ex: District (Andaman and Nico.In.) has pincode 125001 which is of Haryana
# print(plant_df[plant_df['Plant'] == 'X064'])
# invoice_df[invoice_df['District'] == 'Andaman and Nico.In.']

https://blog.projectionhub.com/auto-repair-shop-projected-financials-for-first-3-years/


In [ ]:
# Import label encoder 
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
df = pd.DataFrame()
df['Cust Type'] = label_encoder.fit_transform(invoice_df_2['Cust Type'])
print(df['Cust Type'])
df['Cust Type'] = label_encoder.inverse_transform(df['Cust Type'])
print(df['Cust Type'])

0         6
1         6
2         6
3         6
4         6
         ..
492309    6
492310    6
492311    6
492312    6
492313    6
Name: Cust Type, Length: 492314, dtype: int64
0         Retail
1         Retail
2         Retail
3         Retail
4         Retail
           ...  
492309    Retail
492310    Retail
492311    Retail
492312    Retail
492313    Retail
Name: Cust Type, Length: 492314, dtype: object


In [ ]:
## Plant DF 
print(plant_df.info())
#delCols_With_AnyNaN(plant_df)
print(len(plant_df[(plant_df['Postal Code']<110000) | (plant_df['Postal Code']>930000)]['Postal Code'].unique()))

plant_df[(plant_df['Postal Code']<110000) | (plant_df['Postal Code']>930000)]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 13 columns):
Plant                      438 non-null object
Name 1                     438 non-null object
Valuation Area             438 non-null object
Customer no. - plant       438 non-null object
Vendor number plant        2 non-null object
Factory calendar           438 non-null object
Name 2                     436 non-null object
House number and street    436 non-null object
PO Box                     437 non-null float64
Postal Code                438 non-null int64
City                       438 non-null object
Sales organization         433 non-null object
State                      438 non-null object
dtypes: float64(1), int64(1), object(11)
memory usage: 44.6+ KB
None
1


,Plant,Name 1,Valuation Area,Customer no. - plant,Vendor number plant,Factory calendar,Name 2,House number and street,PO Box,Postal Code,City,Sales organization,State
413,X688,X688 - GIRRAJJI MOTORS,X688,PLX688,NaN,IN,Agra,"Plot No.10K, Mauja Rohta, Gwal",NaN,999999,Agra,MFCD,Uttar Pradesh
414,X689,X689 - POONAM MOTORS,X689,PLX689,NaN,IN,Delhi,"Opp. Rama Krishna Apartment, N",999999.0,999999,Delhi,MFCA,Delhi
420,X706,X706 - ADITYA & BROTHERS,X706,PLX706,NaN,IN,Manendragarh,"Plot No. 343/10, In Front Of K",999999.0,999999,Manendragarh,MFCD,Chhattisgarh
430,X756,X756 - ZENITH AUTOMOTIVE SERVI,X756,PLX756,NaN,IN,Meerut,NaN,0.0,999999,Meerut,MFCD,Uttar Pradesh


In [ ]:

## "Make" Column
make_list = result['Make'].unique().tolist()
print(make_list)

import re
# for each row
for i in range(len(jtd_df)) : 
  print(jtd_df.loc[i, "Description"], df.loc[i, "Age"]) 

shop="hello seattle what have -you (got) 824.72 SY.CYL.HEAD COVER-PLAS"
regex = r'\w+'
list1=re.findall(regex,shop)
print(list1)

print(jtd_df.columns)
print(jtd_df.head(10))
print(jtd_df[jtd_df['Item Category']=='P010']['Description'].unique())

# DF - Identify null rows of Model
result_1 = result[result["Model"].isnull()]["Job Card No"]
print("Total null values of Model : ", len(result_1))
result_1 = pd.merge(result_1, jtd_df, how='left', left_on='Job Card No', right_on='DBM Order', indicator=True)
print("Total null values of Model : ", len(result_1))

# For each of these row's 'Job Card No', find associated Description and add it as a columns Description


# For each row of description
  # fetch words in the sentence
    # fetch eeach word in the list
      # if it contains in the make_list
        # add it to a new column

# Merge the above df back in the result

['GENERAL MOTORS', 'TATA MOTORS', 'MARUTI SUZUKI', 'HYUNDAI', 'TOYOTA', 'HONDA', 'FIAT', 'FORD', 'SKODA', 'MAHINDRA &  MAHINDRA', 'VOLKSWAGEN', 'NISSAN', 'RENAULT', 'MERCEDES BENZ', 'MITSUBISHI MOTORS', 'DAEWOO', 'BMW', 'FORCE', 'SONALIKA', 'VOLVO', 'HINDUSTAN MOTORS', 'AUDI', 'PREMIER\xa0AUTOMOBILES', 'PORCHE', 'LAND ROVER', 'JEEP', 'MORRIS', 'SAN MOTORS']
